In [0]:
from pyspark.sql import functions as F

In [0]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/youtube_playlog.csv"
ACCESS_KEY_ID = "" # cle du compte student
SECRET_ACCESS_KEY = "" # secret key du compte student

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", ACCESS_KEY_ID)
hadoop_conf.set("fs.s3a.secret.key", SECRET_ACCESS_KEY)
hadoop_conf.set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") 

playlog = (spark.read.format('csv')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load(filepath))

In [0]:
playlog.show(5)

+----------+----+-----------+
 timestamp|user| song|
+----------+----+-----------+
1392387533| 0|t1l8Z6gLPzo|
1392387538| 1|t1l8Z6gLPzo|
1392387556| 2|t1l8Z6gLPzo|
1392387561| 3|we5gzZq5Avg|
1392387566| 4|we5gzZq5Avg|
+----------+----+-----------+
only showing top 5 rows

# DATAFRAME ANALYSIS

In [0]:
playlog.schema

Out[9]: StructType(List(StructField(timestamp,IntegerType,true),StructField(user,IntegerType,true),StructField(song,StringType,true)))

In [0]:
playlog.describe().show()

+-------+--------------------+------------------+-------------------+
summary| timestamp| user| song|
+-------+--------------------+------------------+-------------------+
 count| 25739537| 25739537| 25739537|
 mean|1.4427006561045842E9|12697.352275450798|2.532571778181818E8|
 stddev| 3.443284872371195E7|13094.065905828476|8.334645614940468E8|
 min| -139955897| 0| ---AtpxbkaE|
 max| 1554321113| 45903| zzzcFgRMY6c|
+-------+--------------------+------------------+-------------------+

In [0]:
playlog.select([F.count(F.when(F.isnan(c), c).alias(c)) for c in playlog.columns]).show()

+-------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
count(CASE WHEN isnan(timestamp) THEN timestamp END AS `timestamp`)|count(CASE WHEN isnan(user) THEN user END AS `user`)|count(CASE WHEN isnan(song) THEN song END AS `song`)|
+-------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
 0| 0| 0|
+-------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+

In [0]:
playlog == playlog.distinct()

Out[45]: False

*The dataframe playlog have no missing values but seems to have duplicated row*

In [0]:
playlog.count() - playlog.distinct().count()

Out[54]: 123651

In [0]:
df = playlog.groupBy(["song","user","timestamp"]).count()
df_filter = df.filter(df["count"] > 1)
df_filter.describe().show()

+-------+-----------+-----------------+--------------------+-----------------+
summary| song| user| timestamp| count|
+-------+-----------+-----------------+--------------------+-----------------+
 count| 80352| 80352| 80352| 80352|
 mean| null|10491.72020609319|1.4220717379966397E9|2.538866487455197|
 stddev| null|10071.77856615467| 3.277709404678092E7|1.653928906381982|
 min|--Ox7yqyXH8| 0| 1392387642| 2|
 max|zzjQg-JdwTg| 45893| 1554204365| 66|
+-------+-----------+-----------------+--------------------+-----------------+

*There is 123 651 duplicates lines on 25 739 537  lines less than 1%*

In [0]:

if playlog.select(F.col("timestamp" ).cast("float") < 0):
  print("True")

True

In [0]:
playlog.sort("timestamp",ascending = True).show(5)

+----------+----+-----------+
 timestamp|user| song|
+----------+----+-----------+
-139955897| 4|nRa-eGzpT6o|
1392387533| 0|t1l8Z6gLPzo|
1392387537| 70|VJ6ofd0pB_c|
1392387537| 22|Q24VZL8wpOM|
1392387538| 1|t1l8Z6gLPzo|
+----------+----+-----------+
only showing top 5 rows

In [0]:
playlog.filter(playlog["timestamp"] < 0).count()

Out[89]: 1

In [0]:
playlog.filter(playlog["timestamp"] < 0).collect()

Out[90]: [Row(timestamp=-139955897, user=4, song='nRa-eGzpT6o')]

*Only one negative row on all timestamp's column, seems to be an error and we will drop this song*

*we are going to drop duplicated lines and removed the négative value and put it in a new dataframe*

In [0]:
pg_cleaned = playlog.dropDuplicates().filter(playlog["timestamp"] > 0)
pg_cleaned.show(5)

+----------+----+-----------+
 timestamp|user| song|
+----------+----+-----------+
1392387566| 4|we5gzZq5Avg|
1392387538| 1|t1l8Z6gLPzo|
1392387556| 2|t1l8Z6gLPzo|
1392387566| 5|we5gzZq5Avg|
1392387533| 0|t1l8Z6gLPzo|
+----------+----+-----------+
only showing top 5 rows

In [0]:
pg_cleaned.count()

Out[27]: 25615885

#EDA ON TIME FEATURES#

*We are going to transform the timestamp feature on commun date*

In [0]:
df_t = (pg_cleaned.withColumn("datetime", F.from_unixtime("timestamp")).drop('timestamp').orderBy("datetime"))
df_t.select('datetime','user','song').show(5)

+-------------------+----+-----------+
 datetime|user| song|
+-------------------+----+-----------+
2014-02-14 14:18:53| 0|t1l8Z6gLPzo|
2014-02-14 14:18:57| 22|Q24VZL8wpOM|
2014-02-14 14:18:57| 70|VJ6ofd0pB_c|
2014-02-14 14:18:58| 1|t1l8Z6gLPzo|
2014-02-14 14:18:59| 40|d12dJbE_xxw|
+-------------------+----+-----------+
only showing top 5 rows

In [0]:
df_t.select(F.year("datetime").alias('year'), F.month("datetime").alias('month'), F.dayofmonth("datetime").alias('dayofmonth'), F.dayofweek("datetime").alias('dayofweek'), F.dayofyear("datetime").alias('dayofyear'), F.weekofyear("datetime").alias('weekofyear')).show(5)

+----+-----+----------+---------+---------+----------+
year|month|dayofmonth|dayofweek|dayofyear|weekofyear|
+----+-----+----------+---------+---------+----------+
2014| 2| 14| 6| 45| 7|
2014| 2| 14| 6| 45| 7|
2014| 2| 14| 6| 45| 7|
2014| 2| 14| 6| 45| 7|
2014| 2| 14| 6| 45| 7|
+----+-----+----------+---------+---------+----------+
only showing top 5 rows

In [0]:
df_enriched = (df_t.withColumn('year',F.year("datetime")) \
               .withColumn('year',F.year("datetime")) \
               .withColumn('month',F.month("datetime")) \
               .withColumn('dayofmonth',F.dayofmonth("datetime")) \
               .withColumn('dayofweek',F.dayofweek("datetime")) \
               .withColumn('dayofyear',F.dayofyear("datetime")) \
               .withColumn('weekofyear',F.weekofyear("datetime")))
               
df_enriched.select("datetime","user","song","year","month","dayofmonth","dayofweek","dayofyear","weekofyear").show(5)               

+-------------------+----+-----------+----+-----+----------+---------+---------+----------+
 datetime|user| song|year|month|dayofmonth|dayofweek|dayofyear|weekofyear|
+-------------------+----+-----------+----+-----+----------+---------+---------+----------+
2014-02-14 14:18:53| 0|t1l8Z6gLPzo|2014| 2| 14| 6| 45| 7|
2014-02-14 14:18:57| 22|Q24VZL8wpOM|2014| 2| 14| 6| 45| 7|
2014-02-14 14:18:57| 70|VJ6ofd0pB_c|2014| 2| 14| 6| 45| 7|
2014-02-14 14:18:58| 1|t1l8Z6gLPzo|2014| 2| 14| 6| 45| 7|
2014-02-14 14:18:59| 40|d12dJbE_xxw|2014| 2| 14| 6| 45| 7|
+-------------------+----+-----------+----+-----+----------+---------+---------+----------+
only showing top 5 rows

In [0]:
df_enriched.printSchema()

root
-- user: integer (nullable = true)
-- song: string (nullable = true)
-- datetime: string (nullable = true)
-- dayofweek: integer (nullable = true)
-- dayofyear: integer (nullable = true)
-- weekofyear: integer (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- dayofmonth: integer (nullable = true)

In [0]:
display(df_enriched.groupBy("month").count().orderBy('month'))

month,count
1,2003539
2,2147710
3,2518750
4,2265007
5,2366518
6,2086337
7,1953630
8,1760367
9,1966203
10,2386054


In [0]:
display(df_enriched.groupBy("year").count().orderBy('year'))

year,count
2014,8041498
2015,8143052
2016,6017753
2017,2468620
2018,819927
2019,125035


In [0]:
display(df_enriched.groupBy("weekofyear").count().orderBy('weekofyear'))

weekofyear,count
1,434786
2,424200
3,466600
4,496326
5,473586
6,477376
7,505821
8,581735
9,600993
10,568131


In [0]:
display(df_enriched.groupBy("dayofmonth").count().orderBy('dayofmonth'))

dayofmonth,count
1,830349
2,858332
3,820602
4,848713
5,850318
6,871429
7,835162
8,832550
9,842589
10,845356


In [0]:
display(df_enriched.groupBy("dayofyear").count().orderBy('dayofyear'))

dayofyear,count
1,41434
2,48587
3,45183
4,60572
5,53649
6,61874
7,60580
8,62790
9,79198
10,55000
